In [10]:
import pandas as pd
from typing import List
from sqlalchemy import create_engine

In [11]:
base_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green'

files = [
    'green_tripdata_2020-10.csv.gz',
    'green_tripdata_2020-11.csv.gz',
    'green_tripdata_2020-12.csv.gz'
]

In [12]:
def extract(base_url: str, file_names: List[str]) -> pd.DataFrame:
    """Extract data from the given URL and file names."""
    data = []

    for file_name in file_names:
        file_url = f"{base_url}/{file_name}"
        data.append(pd.read_csv(file_url, compression='gzip'))

    return pd.concat(data, ignore_index=True)

In [37]:
def camel_to_snake(name: str) -> str:
    """Convert camelCase names to snake_case."""
    return ''.join(['_'+c.lower() if c.isupper() else c for c in name]).lstrip('_')

# def camel_to_snake(name):
#     """
#     Convert camelCase string to snake_case.
#     """
#     import re

#     # Add an underscore before each uppercase letter and lowercase
#     # the entire string
#     snake_case = re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()
#     return snake_case

In [38]:
camel_to_snake('VendorID')

'vendor_i_d'

In [40]:
df = extract(base_url, files)

df.head()

/tmp/ipykernel_3065/4187983761.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data.append(pd.read_csv(file_url, compression='gzip'))
/tmp/ipykernel_3065/4187983761.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data.append(pd.read_csv(file_url, compression='gzip'))
/tmp/ipykernel_3065/4187983761.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data.append(pd.read_csv(file_url, compression='gzip'))


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2020-10-01 00:31:19,2020-10-01 00:34:55,N,1.0,7,7,1.0,0.79,5.0,0.5,0.5,1.58,0.0,NaN,0.3,7.88,1.0,1.0,0.0
1,2.0,2020-10-01 00:42:12,2020-10-01 00:43:51,N,1.0,179,7,1.0,0.50,4.0,0.5,0.5,0.00,0.0,NaN,0.3,5.30,2.0,1.0,0.0
2,2.0,2020-10-01 00:53:09,2020-10-01 00:55:39,N,1.0,179,223,1.0,0.60,4.0,0.5,0.5,1.06,0.0,NaN,0.3,6.36,1.0,1.0,0.0
3,1.0,2020-10-01 00:12:29,2020-10-01 00:20:08,N,1.0,134,216,2.0,4.40,13.5,0.5,0.5,0.00,0.0,NaN,0.3,14.80,2.0,1.0,0.0
4,1.0,2020-10-01 00:32:38,2020-10-01 00:43:02,N,1.0,82,7,1.0,2.90,10.5,0.5,0.5,0.00,0.0,NaN,0.3,11.80,2.0,1.0,0.0


In [48]:
df.shape

(266855, 20)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266855 entries, 0 to 266854
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               147393 non-null  float64
 1   lpep_pickup_datetime   266855 non-null  object 
 2   lpep_dropoff_datetime  266855 non-null  object 
 3   store_and_fwd_flag     147393 non-null  object 
 4   RatecodeID             147393 non-null  float64
 5   PULocationID           266855 non-null  int64  
 6   DOLocationID           266855 non-null  int64  
 7   passenger_count        147393 non-null  float64
 8   trip_distance          266855 non-null  float64
 9   fare_amount            266855 non-null  float64
 10  extra                  266855 non-null  float64
 11  mta_tax                266855 non-null  float64
 12  tip_amount             266855 non-null  float64
 13  tolls_amount           266855 non-null  float64
 14  ehail_fee              0 non-null   

In [49]:
df_filtered = df[df['passenger_count'] > 0]
df_filtered = df_filtered[df_filtered['trip_distance'] > 0]

df_filtered.shape

(139370, 20)

In [43]:
assert df_filtered[df_filtered['passenger_count'] == 0].empty == True
assert df_filtered[df_filtered['trip_distance'] == 0].empty == True

In [44]:
# Create a new column lpep_pickup_date by converting lpep_pickup_datetime to a date.

df_filtered['lpep_pickup_date'] = pd.to_datetime(
    df_filtered['lpep_pickup_datetime']).dt.date

df_filtered.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,lpep_pickup_date
0,2.0,2020-10-01 00:31:19,2020-10-01 00:34:55,N,1.0,7,7,1.0,0.79,5.0,...,0.5,1.58,0.0,NaN,0.3,7.88,1.0,1.0,0.0,2020-10-01
1,2.0,2020-10-01 00:42:12,2020-10-01 00:43:51,N,1.0,179,7,1.0,0.50,4.0,...,0.5,0.00,0.0,NaN,0.3,5.30,2.0,1.0,0.0,2020-10-01
2,2.0,2020-10-01 00:53:09,2020-10-01 00:55:39,N,1.0,179,223,1.0,0.60,4.0,...,0.5,1.06,0.0,NaN,0.3,6.36,1.0,1.0,0.0,2020-10-01
3,1.0,2020-10-01 00:12:29,2020-10-01 00:20:08,N,1.0,134,216,2.0,4.40,13.5,...,0.5,0.00,0.0,NaN,0.3,14.80,2.0,1.0,0.0,2020-10-01
4,1.0,2020-10-01 00:32:38,2020-10-01 00:43:02,N,1.0,82,7,1.0,2.90,10.5,...,0.5,0.00,0.0,NaN,0.3,11.80,2.0,1.0,0.0,2020-10-01


In [45]:
# Rename columns in Camel Case to Snake Case, e.g. VendorID to vendor_id.

df_renamed = df_filtered.rename(
    columns={'VendorID': 'vendor_id', 'RatecodeID': 'ratecode_id', 'PULocationID': 'pulocation_id', 'DOLocationID': 'dolocation_id'})

df_renamed.head()

,vendor_id,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,ratecode_id,pulocation_id,dolocation_id,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,lpep_pickup_date
0,2.0,2020-10-01 00:31:19,2020-10-01 00:34:55,N,1.0,7,7,1.0,0.79,5.0,...,0.5,1.58,0.0,NaN,0.3,7.88,1.0,1.0,0.0,2020-10-01
1,2.0,2020-10-01 00:42:12,2020-10-01 00:43:51,N,1.0,179,7,1.0,0.50,4.0,...,0.5,0.00,0.0,NaN,0.3,5.30,2.0,1.0,0.0,2020-10-01
2,2.0,2020-10-01 00:53:09,2020-10-01 00:55:39,N,1.0,179,223,1.0,0.60,4.0,...,0.5,1.06,0.0,NaN,0.3,6.36,1.0,1.0,0.0,2020-10-01
3,1.0,2020-10-01 00:12:29,2020-10-01 00:20:08,N,1.0,134,216,2.0,4.40,13.5,...,0.5,0.00,0.0,NaN,0.3,14.80,2.0,1.0,0.0,2020-10-01
4,1.0,2020-10-01 00:32:38,2020-10-01 00:43:02,N,1.0,82,7,1.0,2.90,10.5,...,0.5,0.00,0.0,NaN,0.3,11.80,2.0,1.0,0.0,2020-10-01


In [53]:
df_renamed['vendor_id'].value_counts()

vendor_id
2.0    117408
1.0     21962
Name: count, dtype: int64

In [47]:
# Write your data as Parquet files, partioned by lpep_pickup_date. Use the pyarrow library!

df_renamed.to_parquet('green_tripdata_2020-10_2020-12.parquet',
                      partition_cols=['lpep_pickup_date'], engine='pyarrow')

In [50]:
import os

# Assuming 'output.parquet' is your output directory
output_dir = 'green_tripdata_2020-10_2020-12.parquet'

# Get a list of all directories in the output directory
directories = [name for name in os.listdir(
    output_dir) if os.path.isdir(os.path.join(output_dir, name))]

# The number of directories is the number of partitions
num_partitions = len(directories)

print(f'Number of partitions: {num_partitions}')

Number of partitions: 95
